> В данном примере мы удялем из модели backbone_fpn_head_x_.onnx ноду с именем Conv_1328. Для того что бы выполнить это, нам нужно соеденить вход слоя с его выходом, после чего вызвать для удаляемого слоя метод clear()

In [ ]:
import onnx_graphsurgeon as gs
import numpy as np
import onnx

model = onnx.load("backbone_fpn_head_x_.onnx")
graph = gs.import_onnx(model)
fake_node = [node for node in graph.nodes if node.name == "Conv_1328"][0]

# Get the input node of the fake node
# Node provides i() and o() functions that can optionally be provided an index (default is 0)
# These serve as convenience functions for the alternative, which would be to fetch the input/output
# tensor first, then fetch the input/output node of the tensor.
# For example, node.i() is equivalent to node.inputs[0].inputs[0]
inp_node = fake_node.i()

# Reconnect the input node to the output tensors of the fake node, so that the first identity
# node in the example graph now skips over the fake node.
inp_node.outputs = fake_node.outputs
fake_node.outputs.clear()

# Remove the fake node from the graph completely
graph.cleanup()
onnx.save(gs.export_onnx(graph), "removed.onnx")

> В данном примере мы объединяем выходы модели 2 с входом модели 1. Так как у нас есть слои с одинаковыми названиями мы переменовываем все слои из модели 2. Выходы и входы которые не указаны в io_map остаются свободными.

In [ ]:
import onnx
from onnx import helper, checker

model1 = onnx.load('backbone_fpn_head_x_.onnx')

model2 = onnx.load('backbone_fpn_z.onnx')
checker.check_model(model1)
checker.check_model(model2)

In [ ]:
#Тут мы добовляем префикс в названии модели (т.е m1/ будет добавлено возле каждого имени слоя, иначе если будут слои с одинаковыми именами будет конфликт)
new_model_2 = onnx.compose.add_prefix(model2, prefix='m1_')

In [ ]:
#onnx.save(new_model_2, 'new_model_2.onnx')

In [ ]:
combined_model = onnx.compose.merge_models(
    new_model_2,model1,
    io_map=[('m1_feat_z', 'kernel')]
)

In [ ]:
onnx.save(combined_model, 'combined_model.onnx')

> В данном примере мы показываем как вывести размеры входа и выхода конкретного слоя. Нам это может понадобится для того, что бы создать замену данному слою, если мы захотим его заменить на другой слой (другой слой должен уметь работать с указанными размерностями или иметь перед собоы reshape) 

In [1]:
import onnx_graphsurgeon as gs
import numpy as np
import onnx

model = onnx.load("backbone_fpn_head_x_.onnx")
graph = gs.import_onnx(model)

In [2]:
tensors = graph.tensors()

In [3]:
for index, node in enumerate(graph.nodes):
        if node.name == "Reshape_1327":
            print(index)
            gemm_node = graph.nodes[1327]
            np_w = gemm_node.inputs[1].values

1327


In [4]:
np_w

array([192,   1,   5,   5], dtype=int64)

In [5]:
for index, node in enumerate(graph.nodes):
        if node.name == "Reshape_1326":
            print(index)
            gemm_node = graph.nodes[1326]
            np_w = gemm_node.inputs[1].values

1326


In [6]:
np_w

array([  1, 192,  20,  20], dtype=int64)

In [7]:
for index, node in enumerate(graph.nodes):
        if node.name == "Reshape_1329":
            print(index)
            gemm_node = graph.nodes[1329]
            np_w = gemm_node.inputs[1].values

1329


In [8]:
np_w

array([  1, 192,  20,  20], dtype=int64)

> В данном примере мы показываем как удалить ноду Conv_1328, встаить в место нее другую ноду после чего записать другую ноду в граф. Для того, что бы записать другую ноду мы должны знать где вход и выход новой ноды. Далее, так как оичстка не удаляет вход, мы удаляем вход сами вручную.

In [16]:
import onnx_graphsurgeon as gs
import numpy as np
import onnx

model = onnx.load("backbone_fpn_head_x_.onnx")
graph = gs.import_onnx(model)

for index, node in enumerate(graph.nodes):
        
        if node.name == "Conv_1328":
            print(index)
            gemm_node = graph.nodes[index]
            #np_w = gemm_node.inputs[1].values
            np_w = np.random.rand(192,   1,   5,   5)
            conv_w = gs.Constant(name='W', values=np_w)
            #conv_out_0 = gs.Variable(name='conv_new_0', dtype=np.float32, shape=(192,   1,   5,   5))
            conv_node_0 = gs.Node(op='Conv', name= 'Conv_1000', inputs=[node.inputs[0], conv_w], outputs=[node.outputs[0]],
                                 attrs={
                    'dilations':[1,1],
                    'group':192, 
                    'kernel_shape':[5,5],
                    'pads':[2,2,2,2],
                    'strides':[1,1],
                    
                    }
                                 
                                 
                                 )
            graph.nodes.append(conv_node_0)
            node.outputs.clear()

1328


In [17]:
graph.inputs 

[Variable (img_x): (shape=[1, 3, 320, 320], dtype=float32),
 Variable (kernel): (shape=[1, 192, 5, 5], dtype=float32)]

In [18]:
graph.inputs.pop(1)

Variable (kernel): (shape=[1, 192, 5, 5], dtype=float32)

In [19]:
graph.cleanup()
modified_model = gs.export_onnx(graph)
#onnx.checker.check_model(modified_model)
onnx.save(gs.export_onnx(graph), "removed.onnx")

In [20]:
from onnxsim import simplify

# load your predefined ONNX model
model = onnx.load("removed.onnx")

# convert model
model_simp, check = simplify(model)

assert check, "Simplified ONNX model could not be validated"

ValidationError: Nodes in a graph must be topologically sorted, however input 'onnx::Reshape_2916' of node: 
name: Reshape_1329 OpType: Reshape
 is not output of any previous nodes.